## croplearn.ipynb

In [1]:
import sys
sys.path.insert(0,"../cropseg/")
datasetinfo = { "dataset":"su_african_crops_ghana",
                "groundcollection":"su_african_crops_ghana_labels",
                "s1collection":"su_african_crops_ghana_source_s1",
                "s2collection":"su_african_crops_ghana_source_s2",
                "datadir":"../data/",
                "metadatadir":"../data/metadata/",
                "groundshape":[64,64],
                "s1shape":[64,64],
                "s2shape":[64,64],
                "extension":"tif"
              }
s1bands = [
            {"band":"vv","idx":0},
            {"band":"vh","idx":1},    
          ]  
s2bands = [
            {"band":"blue","idx":0},
            {"band":"green","idx":1},
            {"band":"red","idx":2},
            {"band":"rded1","idx":3},
            {"band":"rded2","idx":4},
            {"band":"rded3","idx":5},
            {"band":"nir","idx":6},
            {"band":"rded4","idx":7},
            {"band":"swir1","idx":8},
            {"band":"swir2","idx":9}
          ]
s1indices = ["vhvv"]
s2indices = ["ndvi","gndvi","gci","rdedci","ndmi"]
erosioniterations = 1
dctcoefficients = 15
skiplist = ["001268","001271"]

In [2]:
import numpy
from osgeo import gdal
import datetime
import scipy.fftpack

from mlhubdata import loadjson
from grounddata import getFieldMasks
from satellitedata import getEOIndicies

def dctcompression(data,dates,ncoefficients,minduration=335):
    if numpy.isnan(numpy.sum(data)) == True or numpy.count_nonzero(data == 0.0) > 5:
        return [0]
    if (dates[len(dates)-1] - dates[0]).days < minduration:
        return [0]
    dct = scipy.fftpack.dct(data,norm="ortho",type=2) 
    return dct[:ncoefficients]

def dataprep(datasetinfo,collection,bands,indices,erosioniterations=1,ncoefficients=6,level="field",skiplist=[],groundshape=[64,64],satelliteshape=[64,64]):
    d = []
    groundmetadata = loadjson(f'{datasetinfo["metadatadir"]}{datasetinfo["groundcollection"]}.json')
    satellitemetadata = loadjson(f'{datasetinfo["metadatadir"]}{collection}.json')
    for i in range(len(groundmetadata)):
        tileid = groundmetadata[i]["id"].split("_")[len(groundmetadata[i]["id"].split("_"))-1]
        if tileid not in skiplist: 
            groundtile = gdal.Open(f'{datasetinfo["datadir"]}{datasetinfo["groundcollection"]}/{groundmetadata[i]["id"]}/labels.{datasetinfo["extension"]}')
            groundtiledata = numpy.array(groundtile.GetRasterBand(1).ReadAsArray(),dtype='int')
            crops = numpy.unique(groundtiledata[groundtiledata != 0])
            fieldmasks = getFieldMasks(groundtiledata,erosioniterations)
            satelliteitems = []
            satellitedates = []
            for j in range(len(satellitemetadata)):
                if tileid in satellitemetadata[j]["id"]:
                    satelliteitems.append(satellitemetadata[j])
            satelliteitems = sorted(satelliteitems,key=lambda k:k["properties"]["datetime"])
            for j in range(len(satelliteitems)):
                satellitedates.append(datetime.datetime.strptime(satelliteitems[j]["properties"]["datetime"],"%Y-%m-%dT%H:%M:%S+0000").date())
            satellitedata = numpy.zeros([len(satelliteitems),len(bands),satelliteshape[0],satelliteshape[1]])
            for j in range(len(satelliteitems)):
                satellitetile = gdal.Open(f'{datasetinfo["datadir"]}{datasetinfo["s1collection"]}/{satelliteitems[j]["id"]}/source.{datasetinfo["extension"]}')    
                for k in range(len(bands)):
                    satellitedata[j][k] = satellitetile.GetRasterBand(k+1).ReadAsArray()
            satelliteindices = getEOIndicies(satellitedata,bands,indices)
            for j in range(len(fieldmasks)):
                for k in range(fieldmasks[j][1]):
                    fieldsatellitedates = []
                    fieldsatellitedata = [[] for _ in range(len(satelliteitems))]
                    for m in range(len(satelliteitems)):
                        fieldsatellitedates.append(satellitedates[m])
                        fm = numpy.copy(fieldmasks[j][0])
                        fieldsatellitedata[m] = satelliteindices[m][0][fm == k+1]
                    fieldsatellitedata = numpy.array(fieldsatellitedata)
                    if level == "pixel":
                        for m in range(len(fieldsatellitedata[0])):
                            values = fieldsatellitedata[:,m]
                            compressioncoefficients = dctcompression(values,fieldsatellitedates,ncoefficients)
                            if compressioncoefficients[0] != 0:
                                compressioncoefficients = numpy.ndarray.tolist(compressioncoefficients)
                                compressioncoefficients.insert(0,crops[j])
                                d.append(compressioncoefficients)
    return d

data = dataprep(datasetinfo,datasetinfo["s1collection"],s1bands,s1indices,erosioniterations=erosioniterations,ncoefficients=dctcoefficients,level="pixel",skiplist=skiplist)
data = numpy.array(data)

In [3]:
d = numpy.copy(data)
numpy.random.shuffle(d)
d[:,0] = d[:,0] - 1
d0 = d[d[:,0] == 0][:40000]
d1 = d[d[:,0] == 1][:40000]
d2 = d[d[:,0] == 2][:40000]
d3 = d[d[:,0] == 3][:40000]
d = numpy.concatenate([d0,d1,d2,d3])
numpy.random.shuffle(d)
xtrain, ytrain = d[4000:,1:],d[4000:,0] 
xtest, ytest = d[:4000,1:],d[:4000,0] 

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
model = tf.keras.Sequential([
    layers.Dense(dctcoefficients*2,activation='relu'),
    layers.Dense(dctcoefficients*2,activation='relu'),
    layers.Dense(4)
])
model.compile(optimizer='Adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=21)
model.summary()

Epoch 1/21
4875/4875 [==============================] - 5s 960us/step - loss: 1.6480 - accuracy: 0.3483
Epoch 2/21
4875/4875 [==============================] - 5s 956us/step - loss: 1.2987 - accuracy: 0.3862
Epoch 3/21
4875/4875 [==============================] - 5s 951us/step - loss: 1.2877 - accuracy: 0.3941
Epoch 4/21
4875/4875 [==============================] - 5s 964us/step - loss: 1.2839 - accuracy: 0.3953
Epoch 5/21
4875/4875 [==============================] - 5s 956us/step - loss: 1.2814 - accuracy: 0.3984
Epoch 6/21
4875/4875 [==============================] - 5s 959us/step - loss: 1.2784 - accuracy: 0.3998
Epoch 7/21
4875/4875 [==============================] - 5s 951us/step - loss: 1.2792 - accuracy: 0.3996
Epoch 8/21
4875/4875 [==============================] - 5s 962us/step - loss: 1.2751 - accuracy: 0.4013
Epoch 9/21
4875/4875 [==============================] - 5s 961us/step - loss: 1.2773 - accuracy: 0.4013
Epoch 10/21
4875/4875 [==============================] - 5s 1ms/

In [5]:
import sklearn.metrics
model.evaluate(xtest,ytest,verbose=1)
predictions = model.predict(xtest)
predict = numpy.argmax(predictions,axis = 1)
true = numpy.array(ytest,dtype=int)
print(sklearn.metrics.f1_score(true,predict,average=None),sklearn.metrics.f1_score(true,predict,average="weighted"))
print(sklearn.metrics.cohen_kappa_score(true,predict))

125/125 [==============================] - 0s 612us/step - loss: 1.2712 - accuracy: 0.4040
[0.45812124 0.15564202 0.4745167  0.42230026] 0.3770130412795332
0.20603506600749366


In [6]:
import sklearn.dummy
dummy = sklearn.dummy.DummyClassifier(strategy="uniform")
dummy.fit(xtrain,ytrain)
predict = dummy.predict(xtest)
true = numpy.array(ytest,dtype=int)
print(sklearn.metrics.accuracy_score(true,predict))
print(sklearn.metrics.f1_score(true,predict,average=None),sklearn.metrics.f1_score(true,predict,average="weighted"))
print(sklearn.metrics.cohen_kappa_score(true,predict))

0.25675
[0.244      0.26013007 0.26821862 0.25481481] 0.2567395848828598
0.009034766363613445
